# Using and understanding Catalogs

<div class="alert alert-info"> <b>NOTE:</b> Catalogs in `xscen` are built upon Datastores in `intake_esm`. For more information on basic usage, such as the `search()` function, please consult their documentation: <a href="https://intake-esm.readthedocs.io/en/stable/">https://intake-esm.readthedocs.io/en/stable/</a>.</div>

Catalogs are made of two files:

- JSON file containing metadata such as the catalog's title, description etc. It also contains an attribute *catalog_file* that points towards the CSV.
- CSV file containing the catalog itself. This file can be zipped.

Two types of catalogs have been implemented in `xscen`.

- __Static catalogs:__ A `DataCatalog` is a *read-only* `intake-esm` catalog that contains information on all available data. Usually, this type of catalog should only be consulted at the start of a new project.

- __Updatable catalogs:__ A `ProjectCatalog` is a *DataCatalog* with additional *write* functionalities. This kind of catalog should be used to keep track of the new data created during the course of a project, such as regridded or bias-corrected data, since it can `update` itself and append new information to the associated CSV file.

__NOTE:__ As to not accidentaly lose data, both catalogs currently have no function to remove data from the CSV file. However, upon initialisation and when updating or refreshing itself, the catalog validates that all entries still exist and, if files have been manually removed, deletes their entries from the catalog.

Catalogs in `xscen` are made to follow a nomenclature that is as close as possible to the Python Earth Science Standard Vocabulary : [https://github.com/ES-DOC/pyessv](https://github.com/ES-DOC/pyessv). The columns are listed below but for more details and concrete examples about the entries, consult [the relevant page in the documentation](../columns.rst):

| Column name | Description |
| :- | :- |
| id | Unique DatasetID generated by `xscen` based on a subset of columns. |
| type | Type of data: [forecast, station-obs, gridded-obs, reconstruction, simulation] |
| processing_level | Level of post-processing reached: [raw, extracted, regridded, biasadjusted] |
| bias_adjust_institution | Institution that computed the bias adjustment. |
| bias_adjust_project | Name of the project that computed the bias adjustment. |
| mip_era | CMIP Generation associated with the data. |
| activity | Model Intercomparison Project (MIP) associated with the data. |
| driving_institution | Institution of the driver model. |
| driving_model | Name of the driver. |
| institution | Institution associated with the source. |
| source | Name of the model or the dataset. |
| experiment | Name of the experiment of the model. |
| member | Name of the realisation (or of the driving realisation in the case of RCMs). |
| xrfreq | Pandas/xarray frequency. |
| frequency | Frequency in letters (CMIP6 format). |
| variable | Variable(s) in the dataset. |
| domain | Name of the region covered by the dataset. |
| date_start | First date of the dataset. |
| date_end | Last date of the dataset. |
| version | Version of the dataset. |
| format | Format of the dataset. |
| path | Path to the dataset. |

Individual projects may use a different set of columns, but those will always be present in the official Ouranos internal catalogs. Some parts of `xscen` will however expect certain column names, so diverging from the official list is to be done with care.

## Basic Catalog Usage

If an official catalog already exists, it should be opened using `xs.DataCatalog` by pointing it to the JSON file:

In [ ]:
from pathlib import Path

from xscen import DataCatalog, ProjectCatalog

output_folder = Path().absolute() / "_data"
output_folder.mkdir(exist_ok=True)


DC = DataCatalog(f"{Path().absolute()}/samples/pangeo-cmip6.json")

DC

The content of the catalog can be accessed by a call to `df`, which will return a `pandas.DataFrame`.

In [ ]:
# Access the catalog
DC.df

The `unique` function allows listing unique elements for either all the catalog or a subset of columns. It can be called in a few various ways, listed below:

In [ ]:
# List all unique elements in the catalog, returns a pandas.Series
DC.unique()

In [ ]:
# List all unique elements in a subset of columns, returns a pandas.Series
DC.unique(["variable", "frequency"])

In [ ]:
# List all unique elements in a single columns, returns a list
DC.unique("id")[0:5]

### Basic .search() commands

The `search` function comes from `intake-esm` and allows searching for specific elements in the catalog's columns. It accepts both wildcards and regular expressions (except for *variable*, which must be exact due to being in *tuples*).

While regex isn't great at inverse matching ("does not contain"), it is possible. Here are a few useful commands:

    - ^string            : Starts with string

    - string$            : Ends with string

    - ^(?!string).*$     : Does not start with string

    - .*(?<!string)$     : Does not end with string

    - ^((?!string).)*$   : Does not contain substring

    - ^(?!string$).*$    : Is not that exact string

This website can be used to test regex commands: https://regex101.com/

In [ ]:
# Regex: Find all entries that start with "rcp"
print(DC.search(experiment="^ssp").unique("experiment"))

In [ ]:
# Regex: Exclude all entries that start with "ssp"
print(DC.search(experiment="^(?!ssp).*$").unique("experiment"))

In [ ]:
# Regex: Find all experiments except the exact string "ssp245"
print(DC.search(experiment="^(?!ssp245$).*$").unique("experiment"))

In [ ]:
# Wildcard: Find all entries that start with NorESM2
print(DC.search(source="NorESM2.*").unique("source"))

It is also possible to search for files that intersect a specific time period.

In [ ]:
DC.search(periods=[["2016", "2017"]]).unique(["date_start", "date_end"])

### Advanced search: xs.search_data_catalogs

`search` has multiple notable limitations for more advanced searches:

- It can't match specific criteria together, such as finding a dataset that would have both 3h precipitation and daily temperature.
- It has no explicit understanding of climate datasets, and thus can't match historial and future simulations together or know how realization members or grid resolutions work.

`xs.search_data_catalogs` was thus created as a more advanced version that is closer to the needs of climate services. It also plays the double role of preparing certain arguments for the extraction function.

Due to how different reference datasets are from climate simulations, this function might have to be called multiple times and the results concatenated into a single dictionary. The main arguments are:

- `variables_and_freqs` is used to indicate which variable and which frequency is required. <b> NOTE:</b> With the exception of fixed fields, where *'fx'* should be used, frequencies here use the `pandas` nomenclature ('D', 'H', '6H', 'MS', etc.).
- `other_search_criteria` is used to search for specific entries in other columns of the catalog, such as *activity*.
- `exclusions` is used to exclude certain simulations or keywords from the results.
- `match_hist_and_fut` is used to indicate that RCP/SSP simulations should be matched with their *historical* counterparts.
- `periods` is used to search for specific time periods.
- `allow_resampling` is used to allow searching for data at higher frequencies than requested.
- `allow_conversion` is used to allow searching for calculable variables, in the case where the requested variable would not be available.
- `restrict_resolution` is used to limit the results to the finest or coarsest resolution available for each source.
- `restrict_members` is used to limit the results to a maximum number of realizations for each source.
- `restrict_warming_level` is used to limit the results to only datasets that are present in the csv used for calculating warming levels.

Note that compared to `search`, the result of `search_data_catalog` is a dictionary with one entry per unique ID. A given unique ID might contain multiple datasets as per `intake-esm`'s definition, because it groups catalog lines per *id - domain - processing_level - xrfreq*. Thus, it separates model data that exists at different frequencies.


#### Example 1: Simple dataset

Let's start by searching for CMIP6 data that has subdaily precipitation, daily temperature and the land fraction data. The main difference compared to searching for reference datasets is that in most cases, `match_hist_and_fut` will be required to match *historical* simulations to their future counterparts. This works for both CMIP5 and CMIP6 nomenclatures.

In [ ]:
import xscen as xs

variables_and_freqs = {"tas": "D", "pr": "3H", "sftlf": "fx"}
other_search_criteria = {"institution": ["NOAA-GFDL"], "experiment": ["ssp585"]}

cat_sim = xs.search_data_catalogs(
    data_catalogs=[f"{Path().absolute()}/samples/pangeo-cmip6.json"],
    variables_and_freqs=variables_and_freqs,
    other_search_criteria=other_search_criteria,
    match_hist_and_fut=True,
)

cat_sim

Two simulations correspond to the search criteria, but as can be seen from the results, it is the same simulation on 2 different grids (`gr1` and `gr2`). If desired, `restrict_resolution` can be called to choose the finest or coarsest grid in such cases.

In [ ]:
variables_and_freqs = {"tas": "D", "pr": "3H", "sftlf": "fx"}
other_search_criteria = {"institution": ["NOAA-GFDL"], "experiment": ["ssp585"]}

cat_sim = xs.search_data_catalogs(
    data_catalogs=[f"{Path().absolute()}/samples/pangeo-cmip6.json"],
    variables_and_freqs=variables_and_freqs,
    other_search_criteria=other_search_criteria,
    match_hist_and_fut=True,
    restrict_resolution="finest",
)

cat_sim

If required, at this stage a dataset can be looked at in more details. If we examine the results (look at the 'date_start' and 'date_end' columns), we'll see that it successfully found historical simulations in the *CMIP* activity and renamed both their *activity* and *experiment* to match the future simulations.

In [ ]:
cat_sim["ScenarioMIP_NOAA-GFDL_GFDL-CM4_ssp585_r1i1p1f1_gr1"].df

#### Example 2: Advanced search

`allow_resampling` and `allow_conversion` are powerful search tools to find data that doesn't explicitely exist in the catalog, but that can easily be computed.

In [ ]:
cat_sim_adv = xs.search_data_catalogs(
    data_catalogs=[f"{Path().absolute()}/samples/pangeo-cmip6.json"],
    variables_and_freqs={"evspsblpot": "D", "tas": "YS"},
    other_search_criteria={"source": ["NorESM2-MM"], "processing_level": ["raw"]},
    match_hist_and_fut=True,
    allow_resampling=True,
    allow_conversion=True,
)
cat_sim_adv

If we examine the SSP5-8.5 results, we'll see that while it failed to find *evspsblpot*, it successfully understood that *tasmin* and *tasmax* can be used to compute it. It also understood that daily *tas* is a valid search result for `{tas: YS}`, since it can be aggregated.

In [ ]:
cat_sim_adv["ScenarioMIP_NCC_NorESM2-MM_ssp585_r1i1p1f1_gn"].unique()

It's also possible to search for multiple frequencies at the same time by using a list of xrfreq.

In [ ]:
cat_sim_adv_multifreq = xs.search_data_catalogs(
    data_catalogs=[f"{Path().absolute()}/samples/pangeo-cmip6.json"],
    variables_and_freqs={"tas": ["D", "MS", "YS"]},
    other_search_criteria={
        "source": ["NorESM2-MM"],
        "processing_level": ["raw"],
        "experiment": ["ssp370"],
    },
    match_hist_and_fut=True,
    allow_resampling=True,
    allow_conversion=True,
)
print(
    cat_sim_adv_multifreq[
        "ScenarioMIP_NCC_NorESM2-MM_ssp370_r1i1p1f1_gn"
    ]._requested_variable_freqs
)

#### Derived variables

The `allow_conversion` argument is built upon `xclim`'s virtual indicators module and `intake-esm`'s [DerivedVariableRegistry](https://ncar.github.io/esds/posts/2021/intake-esm-derived-variables/) in a way that should be seamless to the user. It works by using the methods defined in `xscen/xclim_modules/conversions.yml` to add a registry of *derived* variables that exist virtually through computation methods.

In the example above, we can see that the search failed to find *evspsblpot* within *NORESM2-MM*, but understood that *tasmin* and *tasmax* could be used to estimate it using `xclim`'s `potential_evapotranspiration`.

Most use cases should already be covered by the aforementioned file. The preferred way to add new methods is to [submit a new indicator to xclim](https://xclim.readthedocs.io/en/stable/contributing.html), and then to add a call to that indicator in `conversions.yml`. In the case where this is not possible or where the transformation would be out of scope for `xclim`, the calculation can be implemented into `xscen/xclim_modules/conversions.py` instead.

Alternatively, if other functions or other parameters are required for a specific use case (e.g. using `relative_humidity` instead of `relative_humidity_from_dewpoint`, or using a different formula), then a custom YAML file can be used. This custom file can be referred to using the `conversion_yaml` argument of `search_data_catalogs`.

`.derivedcat` can be called on a catalog to obtain the list of DerivedVariable and the function associated to them. In addition, `._requested_variables` will display the list of variables that will be opened by the `to_dataset_dict()` function, including *DerivedVariables*.

**NOTE:** `_requested_variables` should NOT be modified under any circumstance, as it is likely to make `to_dataset_dict()` fail. To add some transparency on which variables have been *requested* and which are the *dependent* ones, `xscen` has added `_requested_variables_true` and `_dependent_variables`. This is very likely to be changed in the future.

In [ ]:
cat_sim_adv["ScenarioMIP_NCC_NorESM2-MM_ssp585_r1i1p1f1_gn"].derivedcat

In [ ]:
print(cat_sim_adv["ScenarioMIP_NCC_NorESM2-MM_ssp585_r1i1p1f1_gn"]._requested_variables)
print(
    f"Requested: {cat_sim_adv['ScenarioMIP_NCC_NorESM2-MM_ssp585_r1i1p1f1_gn']._requested_variables_true}"
)
print(
    f"Dependent: {cat_sim_adv['ScenarioMIP_NCC_NorESM2-MM_ssp585_r1i1p1f1_gn']._dependent_variables}"
)

<div class="alert alert-warning"> <b>WARNING:</b> Note that `allow_conversion`  currently fails if:
<ul>
<li>The requested DerivedVariable also requires a DerivedVariable itself.</li>
<li>The dependent variables exist at different frequencies (e.g. 'pr @1hr' & 'tas @3hr')</li>
</ul>
</div>

## Creating a New Catalog from a Directory

### Initialisation

The `create` argument of `ProjectCatalog` can be called to create an empty *ProjectCatalog* and a new set of JSON and CSV files. The JSON file follows the ESM Catalog Specification v.0.1.0: https://github.com/NCAR/esm-collection-spec

By default, `xscen` will populate the JSON with generic information, defined in `catalog.esm_col_data`. That metadata can be changed using the `project` argument with entries compatible with the ESM Catalog Specification (refer to the link above). Usually, the most useful and common entries will be: 

- title
- description

`xscen` will also instruct `intake_esm` to group catalog lines per *id - domain - processing_level - xrfreq*. This should be adequate for most uses. In the case that it is not, the following can be added to `project`:

- "aggregation_control": {"groupby_attrs": [list_of_columns]}

Other attributes and behaviours of the project definition can be modified in a similar way.

In [ ]:
project = {
    "title": "tutorial-catalog",
    "description": "Catalog for the tutorial NetCDFs.",
}

PC = ProjectCatalog(
    str(output_folder / "tutorial-catalog.json"),
    create=True,
    project=project,
    overwrite=True,
)

In [ ]:
# The metadata is stored in PC.esmcat
PC.esmcat

### Appending new data to a ProjectCatalog

At this stage, the CSV is still empty. There are two main ways to populate a catalog with data:

- Using `xs.ProjectCatalog.update_from_ds` to append a Dataset and populate the catalog columns using metadata. 

- Using `xs.catalog.parse_directory` to parse through existing NetCDF or Zarr data and decode their information based on file and directory names.

This tutorial will focus on `catalog.parse_directory`, as `update_from_ds` is moreso a function that will be called during a climate-scenario-generation workflow. See the [Getting Started](2_getting_started.ipynb#Updating-the-catalog) tutorial for more details on `update_from_ds`.

#### Parsing a directory 

<div class="alert alert-info"> <b>NOTE:</b> If you are an Ouranos employee, this section should be of limited use (unless you need to retroactively parse a directory containing exiting datasets). Please consult the existing Ouranos catalogs using xs.search_data_catalogs instead.</div>

The `parse_directory` function relies on analyzing patterns to adequately decode the filenames to store that information in the catalog. 

- Patterns are a succession of column names in curly brackets. See below for an example. The pattern starts where the directory path stops.
- The `parallel_depth` argument can be used to change the level at which to parallelize the file (and thus the `globpattern`) search. A value of 1 (default and minimum), means the subfolders of each directory are searched in parallel, a value of 2 would search the subfolders' subfolders in parallel, and so on.
- If necessary, `read_from_file` can be used to open the files and read metadata from global attributes. Refer to the API for Docstrings and usage.
- In cases where some column information is the same across all data, `homogenous_info` can be used to explicitely give an attribute to the datasets being processed.
- Anything that isn't filled will be marked as `None`. 

In [ ]:
from xscen.catalog import parse_directory

df = parse_directory(
    directories=["samples/tutorial/"],
    globpattern="*.nc",
    patterns=[
        "{activity}/{domain}/{institution}/{source}/{experiment}/{member}/{frequency}/{*}.nc"
    ],
    parallel_depth=1,
    homogenous_info={
        "mip_era": "CMIP6",
        "type": "simulation",
        "processing_level": "raw",
    },
    read_from_file=["variable", "date_start", "date_end"],
)
df

#### Unique Dataset ID

In addition to the parse itself, `catalog.parse_directory` will create a unique Dataset ID that can be used to easily determine one simulation from another. This can be edited with the `id_columns` argument of `parse_directory`, but by default, IDs are based on CMIP6's ID structure with additions related to regional models and bias adjustment:

- `{bias_adjust_project} _ {mip_era} _ {activity} _ {driving_model} _ {institution} _ {source} _ {experiment} _ {member} _ {domain}`

This utility can also be called by itself through `xs.catalog.generate_id()`.

**NOTE:** Note that when constructing IDs, empty columns will be skipped.

In [ ]:
df.iloc[0]["id"]

#### Appending data using ProjectCatalog.update()

At this stage, `df` is a `pandas.DataFrame`. `ProjectCatalog.update` is used to append this data to the CSV file and save the results on disk.

In [ ]:
PC.update(df)

PC